# Parsing Madness:
## Analyzing linguistic and thematic patterns in QAnon 'drops'
## An exercise in web scraping, dataframe creation and management, and natural language processing
##### beautiful soup -> pandas dataframe
##### request -> soup -> imgs and text -> pandas dataframe
##### text -> cleaned -> tokenized -> remove stop words -> features dictionary

In [4]:
import matplotlib.pyplot as plt
import matplotlib.pyplot as mpim
import requests
import itertools
import re
import string 
import nltk
import os
import shutil
import urllib.request
import pandas as pd
import numpy as np
import datetime as dt
import pickle
import networkx as nx

from string import digits
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.decomposition import NMF, LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from bs4 import BeautifulSoup
from itertools import islice, zip_longest, chain 
from skimage import io
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
from nltk import RegexpParser, Tree
from nltk.util import ngrams
from urllib.request import Request, urlopen
from IPython.display import Image
from collections import Counter

stop_words = set(stopwords.words('english'))
punct = string.punctuation + str('’') + str('“') + str('”') + str('‘') + str('–') + str('…')

normalizer = WordNetLemmatizer()

def get_part_of_speech(word):
  probable_part_of_speech = nltk.corpus.wordnet.synsets(word)
  pos_counts = Counter()
  pos_counts["n"] = len(  [ item for item in probable_part_of_speech if item.pos()=="n"]  )
  pos_counts["v"] = len(  [ item for item in probable_part_of_speech if item.pos()=="v"]  )
  pos_counts["a"] = len(  [ item for item in probable_part_of_speech if item.pos()=="a"]  )
  pos_counts["r"] = len(  [ item for item in probable_part_of_speech if item.pos()=="r"]  )
  most_likely_part_of_speech = pos_counts.most_common(1)[0][0]
  return most_likely_part_of_speech

def preprocess_text(text):
  cleaned = re.sub(r'\W+', ' ', text).lower()
  tokenized = nltk.word_tokenize(cleaned)
  normalized = " ".join([normalizer.lemmatize(token, get_part_of_speech(token)) for token in tokenized if token not in stop_words])
  return normalized

def text_to_bow(some_text):
    bow_dictionary = {}
    for text in some_text:
        if text in bow_dictionary:
            bow_dictionary[text] += 1
        else:
            bow_dictionary[text] = 1
    return bow_dictionary

In [5]:
os.getcwd()

'/Users/kylereaves/Documents/GitHub/parsing_madness'

beautifulsoup scraping

- request 
- soup object for each request object 
- traversing html structure to retrieve image, text, and urls 
- problems: some of the the images for the metadata are repeated as text in the many body

## pandas dataframe

In [6]:
q_df = pd.read_csv('q_drops.txt')
q_df.drop(columns=['Unnamed: 0'])

with open('drops.pickle', 'rb') as f:
    drops_pickled = pickle.load(f)

images_from_pickle = re.findall(r'https://qposts\S+(?:jpg|jpeg|png)', str(drops_pickled))
links_from_pickle = re.findall(r'\bhttps?://(?!\S+(?:jpe?g|png))\S+', str(drops_pickled))

In [21]:
def drop_and_len(i):
    if i > len(drops_pickled) - 1:
        return "{0} hasn't dropped yet.".format(i)
    return drops_pickled[i], len(drops_pickled[i])

In [5]:
pickled_only_text = []
for drop in drops_pickled:
    inner = ''
    for line in drop:
        if line.startswith('https') or line.startswith('>>') or line == 'Q' or line.startswith('>'):
            pass
        else:
            #line = re.sub('[%s]' % re.escape(punct), '', line)
            line = re.sub(r'\n', ' ', line)
            line = re.sub(r'https?\w+', ' ', line)
            line = re.sub(r'\d(th)?', ' ', line)
            #line = re.sub(r'\u2002', ' ', line)
            line = re.sub(r'www\w+', ' ', line)
            line = line.lower()
            if not line:
                pass
            else:
                inner += ' '.join(line.split())
    pickled_only_text.append(inner)

In [6]:
# with open('cleaned_text.pickle', 'wb') as f:
#   pickle.dump(pickled_only_text, f, pickle.HIGHEST_PROTOCOL)

regex to remove hyperlinks without capturing other ' , or ]

In [7]:
regex = r'https\S+[\w$]'

In [8]:
outter_list = []
for i in range(0, len(q_df.q_drop)):
    inner = []
    text = q_df.q_drop[i]
    text = re.sub(regex, '', text)
    text = re.sub(r'>>\S+[png$]', '', text)
    text = re.sub(r'>>\S+[\d$]', '', text)
    text = re.sub('\[', '', text)
    text = re.sub('\]', '', text)
    text = re.sub('/', ' ', text)
    text = re.sub('\?', '', text)
    text = re.sub('\(', '', text)
    text = re.sub('\)', '', text)
    text = re.sub('\!', '', text)
    text = re.sub('>', '', text)
    text = re.sub('…', '', text)
    text = re.sub(r'\u2002\u2002', ' ', text)
    text = re.sub('\.', '', text)
    text = re.sub('\*', '', text)
    text = re.sub('_', ' ', text)
    text = re.sub('C19', 'covid', text)
    text = re.sub('\$', '', text)
    text = re.sub(r'\d\S+', '', text)
    text = text.lower()
    outter_list.append(text)

### this method parses the first character of the first word of the first line of the first drop
### returns a rebuilt list
### to do:
*perform the same operation for the range of drops_pickled*
-
**perform the same operation for the range of the drops_pickled**

In [9]:
re.sub('\[', '', q_df.q_drop[0])

"'https://www.youtube.com/watch?v=O1l-nR1Apj4', 'Q']"

In [10]:
no_brackets = []
for i in range(0, len(q_df)):
    drop = q_df.q_drop[i]
    text = re.sub('\[', '', drop)
    text = re.sub('\]', '', text)
    no_brackets.append(text)

- This following drop incurred several instances of unintentional concatenation
- take all of the regex substitution from the for loops and transfer them into the function
- try out our refined cleaning function to see if those problems have been resolved

In [23]:
drops_pickled[-9][0]

'Projection  D’s can’t lose control over the black population. At some point the great awakening will occur whereby these false local / national black leaders are corrupt and paid off to help keep the black pop poor and in need.  D’s formed the confederate states against freeing slaves. D’s formed the KKK. HRC’s mentor is who? What happens if the truth about Haiti is released? Do D’s lose majority of the vote? Through the looking glass. They rely on the MSM to keep the narrative going but tech is entrenching on their controls. They missed this in 2016 and desperately attempting to censor now due to CIA cash infusions. This will fail.'

In [134]:
def drop_cleaning(drop):
    item_list = drop.split()
    sent_tokens = nltk.sent_tokenize(drop)
    num_of_sents = len(sent_tokens)
    
    cleaned_sents = []
    for sent in sent_tokens:
        sent = re.sub('\.', '', sent)
        sent = re.sub('\?', '', sent)
        sent = re.sub('\/', '', sent)
        sent = re.sub('\(', '', sent)
        sent = re.sub('\)', '', sent)
        sent = re.sub('\[', '', sent)
        sent = re.sub('\]', '', sent)
        sent = re.sub('\*', '', sent)
        sent = re.sub('&', 'and', sent)
        sent = re.sub('2grand', 'two grand', sent)
        sent = re.sub('\_', '', sent)
        sent = re.sub('\'', '', sent)
        sent = re.sub(r'>>\d\S+(\d)', '', sent)
        sent = re.sub(r'http\S+\b', '', sent)
        sent = re.sub('\-', ' ', sent)
        sent = re.sub(r':\d\S', '', sent)
        sent = re.sub('\#', '', sent)
        sent = re.sub('\,', '', sent)
        sent = re.sub('\:', '', sent)
        sent = re.sub('\+', '', sent)
        
        #can this all be avoided with the bizare regex from that ladies NLP tutorial?
        
        # splitting the sentence again
        # and then joining it together again
        # removes trailing spaces
        sent = sent.lower().split()
        sent = ' '.join(sent)
        cleaned_sents.append(sent)
    
    return cleaned_sents
    #return item_list

In [136]:
drop_cleaning(drops_pickled[100][0])

['wife cia']

In [140]:
drop_cleaning(drops_pickled[4000][0])

['how bad is the corruption',
 'fbi pastpresent 1 1 2 29 16 doj pastpresent 1 1 2 18 state pastpresent 1 1 41 removal is the least of their problems',
 'projection',
 'russia>dhrc twitter bots>goog operated not russianarrative and political slant biden china',
 'big development',
 'traitors everywhere',
 'america for sale',
 'flynn',
 'targeted',
 'why',
 'who knows where the bodies are buried',
 'cleared of all charges',
 'trump admin v2',
 'election theft',
 'last hope',
 'congressional focus',
 'impeach',
 'they think you are stupid',
 'they think you will follow the stars',
 'they openly call you sheepcattle',
 'there will come a time none of them will be able to walk down the street',
 'biggest fear',
 'public awakening',
 'q']

In [103]:
# the range of drops pickled:
# then go for the length of every drop that's been split

for i in range(0, len(drops_pickled)):
    print(i, len(drops_pickled[i]), drops_pickled[i])

0 2 ['https://www.youtube.com/watch?v=O1l-nR1Apj4', 'Q']
1 2 ['Durham.', 'Q']
2 14 ['Shall we play a game?', '[N]othing [C]an [S]top [W]hat [I]s [C]oming', 'NCSWIC', 'https://www.cisa.gov/safecom/NCSWIC', 'Who stepped down today [forced]?', 'https://www.cisa.gov/bryan-s-ware', 'More coming?', 'Why is this relevant?', "How do you 'show' the public the truth?", "How do you 'safeguard' US elections post-POTUS?", "How do you 'remove' foreign interference and corruption and install US-owned voter ID law(s) and other safeguards? ", 'It had to be this way.', 'Sometimes you must walk through the darkness before you see the light. ', 'Q']
3 4 ['Nothing can stop what is coming.', 'Nothing!', 'Q', 'https://qposts.online/assets/images/63d961c72b558a8009823025001eebe46be8903b830cebca81dc7a27a7e4025c.png']
4 5 ['https://www.youtube.com/watch?v=9tjdswqGGVg&list=PLr4E97YzUfEMIr23hzOBmk9mg-z8TDOXj', 'that this nation, under God, shall have a new birth of freedom  and that government of the people, by t

#### each drop is broken down into strings, 
#### each string is broken down into words, 
#### each word is stripped of extraneous characters
#### each word is then joined into a string, each string is joined together
#### one drop, one string

- append each drop to a master list 

In [163]:
', '.join(drops_pickled[2]).splitlines()

["Shall we play a game?, [N]othing [C]an [S]top [W]hat [I]s [C]oming, NCSWIC, https://www.cisa.gov/safecom/NCSWIC, Who stepped down today [forced]?, https://www.cisa.gov/bryan-s-ware, More coming?, Why is this relevant?, How do you 'show' the public the truth?, How do you 'safeguard' US elections post-POTUS?, How do you 'remove' foreign interference and corruption and install US-owned voter ID law(s) and other safeguards? , It had to be this way., Sometimes you must walk through the darkness before you see the light. , Q"]

In [277]:
for i in range(1, len(drops_pickled)):
    for item in drops_pickled[i]:
        total_words = item.split()
        word_count = len(total_words)
        tokens = []
        new_drop = []
        string = []
        for word in total_words:
            if word[0] == '>':
                continue
            if word[0] in digits:
                continue
            if word[0] == '-':
                continue
            if word[0] == '@':
                continue
            elif word.startswith('http'):
                continue

            else:
                new_drop.append(word.lower())
        for drop in new_drop:
            cleaned = []
            text = re.sub('\)', '', drop)
            text = re.sub('\(', '', text)
            text = re.sub('\?', '', text)
            text = re.sub('\^', '', text)
            text = re.sub('\/', '', text)
            text = re.sub('\!', '', text)
            text = re.sub('_', '', text)
            text = re.sub('\:', '', text)
            text = re.sub('\[', '', text)
            text = re.sub('\]', '', text)
            if text.startswith('.'):
                continue
            else:
                one_string = ' '.join(cleaned)
                cleaned.append(text.strip('.').strip(','))
            tokens.append(' '.join(cleaned))

        string.append(' '.join(tokens))


        #print(sentences)

In [79]:
drop_and_len(3500)

([">>1699813>>1699764 2grand\u2002says\u2002IG\u2002report\u2002loaded\u2002top\u2002to\u2002bottom\u2002with\u2002redactions.\u2002 You\u2002might\u2002get\u2002to\u2002see\u2002a\u2002couple\u2002'a'\u2002and\u2002'they'\u2002and\u2002'the'\u2002but\u2002after\u2002that.\u2002\u2002*CENSORED*  There will be many redactions. Why was the EO put on ice? To wait for the release of the 'Clinton Investigation' portion of the IG report? Why? To include in the DECLAS? Why would it be important to include? (all in one) Optics are meaningful. Political hit job narrative. R's v D's. Not right v wrong. Projection. END OF THE D PARTY [leaders]. IG>Huber. Who appointed Huber? Re_read (again). Slowly & carefully.  http://www.breitbart.com/big-government/2018/03/31/turley-sessions-using-utah-federal-prosecutor-much-better-trump-2nd-special-counsel/ Q "],
 1)

In [280]:
string[-1]

'hillary clinton will be arrested between am am est on monday the morning on oct hrc extradition already in motion effective yesterday with several countries in case of cross border run passport approved to be flagged effective expect massive riots organized in defiance and others fleeing the us to occur us m’s will conduct the operation while ng activated proof check locate a ng member and ask if activated for duty across most major cities'

In [211]:
' '.join(string[0].split())

'hillary clinton will be arrested between am am est on monday the morning on oct hrc extradition already in motion effective yesterday with several countries in case of cross border run passport approved to be flagged effective expect massive riots organized in defiance and others fleeing the us to occur us m’s will conduct the operation while ng activated proof check locate a ng member and ask if activated for duty across most major cities'

In [213]:
for word in q_df.q_drop[150].splitlines():
    word = re.sub(r'[^ ]+\.[^ ]+', '', word)
    word = word.lower()
    word = word.replace('[', '')
    word = word.replace(']', '')
    word = word.replace('?', '')
    word = word.replace('(', '')
    word = word.replace(')', '')
    word = word.replace('.', '')
    sentence = ''.join(word)
    print(sentence.strip())

'fake/doctored image/video dropped here re: epstein cell cam explicity designed to ban - terminate any twit users  'do not take the  'use discernment ', 'q'


In [257]:
len(drop_and_len(600)[0])

35

since this function returns a tuple, I'm going to access it like I would an item in a list

In [138]:
drop_and_len(3500)[0][0].split()

['>>1699813>>1699764',
 '2grand',
 'says',
 'IG',
 'report',
 'loaded',
 'top',
 'to',
 'bottom',
 'with',
 'redactions.',
 'You',
 'might',
 'get',
 'to',
 'see',
 'a',
 'couple',
 "'a'",
 'and',
 "'they'",
 'and',
 "'the'",
 'but',
 'after',
 'that.',
 '*CENSORED*',
 'There',
 'will',
 'be',
 'many',
 'redactions.',
 'Why',
 'was',
 'the',
 'EO',
 'put',
 'on',
 'ice?',
 'To',
 'wait',
 'for',
 'the',
 'release',
 'of',
 'the',
 "'Clinton",
 "Investigation'",
 'portion',
 'of',
 'the',
 'IG',
 'report?',
 'Why?',
 'To',
 'include',
 'in',
 'the',
 'DECLAS?',
 'Why',
 'would',
 'it',
 'be',
 'important',
 'to',
 'include?',
 '(all',
 'in',
 'one)',
 'Optics',
 'are',
 'meaningful.',
 'Political',
 'hit',
 'job',
 'narrative.',
 "R's",
 'v',
 "D's.",
 'Not',
 'right',
 'v',
 'wrong.',
 'Projection.',
 'END',
 'OF',
 'THE',
 'D',
 'PARTY',
 '[leaders].',
 'IG>Huber.',
 'Who',
 'appointed',
 'Huber?',
 'Re_read',
 '(again).',
 'Slowly',
 '&',
 'carefully.',
 'http://www.breitbart.com/big

In [248]:
len(drop_and_len(600)[0])

35

In [244]:
len(nltk.sent_tokenize(drop_and_len(600)[0][0]))


10

In [1]:
for i in range(1, len(drops_pickled)):
    for string in drops_pickled[i][0:]:
        inner_list = []
        inner_list.append(''.join(string))
        print(list(itertools.chain.from_iterable(inner_list)))

NameError: name 'drops_pickled' is not defined

In [140]:
with open('cleaned_text.pickle', 'rb') as f:
    cleaned_text = pickle.load(f)

In [12]:
output = []
for sentence in cleaned_text:
    inner = []
    for word in sentence.split():
        if word and word not in stop_words:
            inner.append(word)
    output.append(' '.join(inner))

In [13]:
q_concacted_str = '. '.join(cleaned_text)

In [14]:
stops_removed = [out for out in output if out]

In [15]:
q_counter = CountVectorizer()

q_matrix = q_counter.fit_transform(stops_removed)

q_features = q_counter.get_feature_names()

q_freq_df = pd.DataFrame(q_matrix.T.todense(), index=q_features)

In [16]:
q_freq_df.sort_values(by=q_freq_df.columns[-1], ascending=False)

,0,1,2,3,4,5,6,7,8,9,...,12063,12064,12065,12066,12067,12068,12069,12070,12071,12072
potus,0,0,0,0,0,0,0,0,0,0,...,2,0,0,2,0,4,0,4,3,4
military,0,0,0,0,0,0,0,0,0,0,...,2,0,0,1,0,1,0,1,0,3
soros,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,3,2,1,0,2
hrc,0,0,0,0,0,0,0,0,0,0,...,0,0,0,2,2,0,0,0,1,2
conditions,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
federation,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
feds,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
feed,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
feeder,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
q_tokens = [nltk.word_tokenize(text) for text in cleaned_text]
q_tagged = [nltk.pos_tag(token) for token in q_tokens]

In [18]:
NUM_TOPICS = 10

vectorizer = CountVectorizer(min_df=5, max_df=0.9, stop_words='english', lowercase=True)
data_vectorized = vectorizer.fit_transform(stops_removed)

lda_model = LatentDirichletAllocation(n_components=NUM_TOPICS, max_iter=10, learning_method='online')
lda_Z = lda_model.fit_transform(data_vectorized)

nmf_model = NMF(n_components=NUM_TOPICS)
nmf_Z = nmf_model.fit_transform(data_vectorized)

lsi_model = TruncatedSVD(n_components=NUM_TOPICS)
lsi_Z = lsi_model.fit_transform(data_vectorized)

def print_topics(model, vectorizer, top_n=10):
    for idx, topic in enumerate(model.components_):
        print("Topic %d:" % (idx))
        print([(vectorizer.get_feature_names()[i], topic[i])
        for i in topic.argsort()[:-top_n - 1:-1]])

In [19]:
print("LDA Model:")
print_topics(lda_model, vectorizer)
print("=" * 20)

LDA Model:
Topic 0:
[('think', 475.4546334165186), ('president', 191.70922845897235), ('follow', 182.98457072148815), ('world', 168.75126259196816), ('coming', 166.6714894564681), ('states', 157.43228993456924), ('united', 154.8755938716277), ('republican', 128.9995724148232), ('divided', 127.63300078967917), ('america', 113.14632196544625)]
Topic 1:
[('public', 380.8690652836086), ('bank', 234.20650388375282), ('truth', 210.66106253692934), ('define', 188.23082224079567), ('treason', 179.16454778087927), ('prevent', 161.65519366378905), ('central', 134.9844272539878), ('hussein', 131.72471313092345), ('events', 110.20795545531416), ('light', 99.94721458298396)]
Topic 2:
[('people', 457.3037778160853), ('time', 265.76906572702495), ('god', 211.1638716062722), ('new', 187.75056948225694), ('happens', 143.2735341362532), ('political', 138.38813524825073), ('order', 123.28362814500073), ('sessions', 119.39518714153989), ('war', 105.23634383079869), ('government', 104.10890332276101)]
Topi

In [20]:
print("LSI Model:")
print_topics(lsi_model, vectorizer)
print("=" * 20)

LSI Model:
Topic 0:
[('bank', 0.8725848340652119), ('central', 0.439200757941844), ('national', 0.12941947044732277), ('states', 0.08637449096116853), ('reserve', 0.0823807525375856), ('republic', 0.07173768042409012), ('west', 0.043642317052234286), ('authority', 0.03778848415834826), ('monetary', 0.03772641227152957), ('new', 0.03140308929340845)]
Topic 1:
[('republican', 0.8672415719140729), ('democrat', 0.31789900231747764), ('house', 0.2612278915116428), ('john', 0.0857208654366074), ('potus', 0.08195329517495725), ('senate', 0.0819473051077891), ('ryan', 0.05045518903910443), ('bob', 0.049568131357534954), ('jr', 0.04942226140841578), ('important', 0.04298433927246466)]
Topic 2:
[('potus', 0.5603981082362045), ('relevant', 0.22280830485755046), ('sa', 0.20005102784533435), ('people', 0.18773788547502435), ('good', 0.16253155255567098), ('think', 0.16242080091491978), ('news', 0.1296568050524482), ('impeach', 0.1261954525803195), ('define', 0.10654700490576882), ('power', 0.105542

In [21]:
print("NMF Model:")
print_topics(nmf_model, vectorizer)
print("=" * 20)

NMF Model:
Topic 0:
[('bank', 8.292930869051586), ('central', 4.1711935194198935), ('national', 1.2220240447199209), ('states', 0.8109472653665443), ('reserve', 0.782490151749852), ('republic', 0.6812023221537348), ('west', 0.4147764396559338), ('monetary', 0.3585483550779582), ('authority', 0.3575475704266016), ('new', 0.28409295807666796)]
Topic 1:
[('republican', 6.752527798849936), ('democrat', 2.475289579116315), ('house', 1.9852413482787048), ('john', 0.603710195743942), ('senate', 0.5939927502773127), ('ryan', 0.3865791957209088), ('jr', 0.3834378335107594), ('bob', 0.3824853327905929), ('gowdy', 0.28484524576086034), ('committee', 0.20422155121528932)]
Topic 2:
[('potus', 6.78198500843284), ('good', 2.9597458601105875), ('impeach', 2.7048509264995895), ('sc', 0.503022439724751), ('believe', 0.49536429427114487), ('fight', 0.48865425835711407), ('twitter', 0.4732103476847585), ('mueller', 0.45653129957994926), ('military', 0.4501600077114966), ('attack', 0.35805332030980863)]
To

In [22]:
unseen_text = 'the idea of freedom is impossible of realization because no one knows how to use it with moderation.'
x = nmf_model.transform(vectorizer.transform([unseen_text]))[0]

In [3]:
document_1 = 'It was only in the Reich itself that the "chosen ones" saw nothing of all this. As if stricken with blindness, they walked by the side of the corpse, and in the indications of decomposition they thought they detected signs of "new" life.'

document_2 = 'Out of the temporary evil we are now compelled to commit will emerge the good of an unshakable rule, which will restore the regular course of the machinery of the national life, brought to naught by liberalism. The result justifies the means. Let us, however, in our plans, direct our attention not so much to what is good and moral as to what is necessary and useful.'

document_3 = 'Just as Nature concentrates, not on safeguarding that which exists, but on breeding the coming generation as the representative of the species, so in human life it is less a question of artificially cultivating the existing evils which, human nature being what it is, would be ninety-nine per cent impossible, but rather to assure healthier paths for future development from the start.'

corpus = [document_1, document_2, document_3]

processed_corpus = [preprocess_text(doc) for doc in corpus]

tfidf_vectorizer = TfidfVectorizer(norm=None)

tf_idf_scores = tfidf_vectorizer.fit_transform(processed_corpus)

feature_names = tfidf_vectorizer.get_feature_names()
corpus_index = [n for n in processed_corpus]

df_tf_idf = pd.DataFrame(tf_idf_scores.T.todense(), index=feature_names, columns=corpus_index)

NameError: name 'preprocess_text' is not defined

In [24]:
df_tf_idf

,reich chosen one saw nothing stricken blindness walk side corpse indication decomposition think detect sign new life,temporary evil compel commit emerge good unshakable rule restore regular course machinery national life bring naught liberalism result justify mean let u however plan direct attention much good moral necessary useful,nature concentrate safeguard exist breeding come generation representative specie human life less question artificially cultivate exist evil human nature would ninety nine per cent impossible rather assure healthy path future development start
artificially,0.000000,0.000000,1.693147
assure,0.000000,0.000000,1.693147
attention,0.000000,1.693147,0.000000
blindness,1.693147,0.000000,0.000000
breeding,0.000000,0.000000,1.693147
...,...,...,...
think,1.693147,0.000000,0.000000
unshakable,0.000000,1.693147,0.000000
useful,0.000000,1.693147,0.000000
walk,1.693147,0.000000,0.000000


In [2]:
flat = list(itertools.chain.from_iterable(q_tokens))
flat_stops_removed = [f for f in flat if f not in stop_words]

count_vec = CountVectorizer()

doc_2_processed = preprocess_text(document_2)

term_frequencies = count_vec.fit_transform([doc_2_processed])
tf_feature_names = count_vec.get_feature_names()

df_term_frequencies = pd.DataFrame(term_frequencies.T.todense(), index=tf_feature_names, columns=['Term Frequency'])

NameError: name 'itertools' is not defined

In [26]:
df_term_frequencies

,Term Frequency
attention,1
bring,1
commit,1
compel,1
course,1
direct,1
emerge,1
evil,1
good,2
however,1


In [27]:
nbrs = NearestNeighbors(n_neighbors=10).fit(tf_idf_scores)

def closest_nbrs(word):
    row = df_tf_idf.index.get_loc(word)
    distances, indices = nbrs.kneighbors(tf_idf_scores.getrow(row))
    words_similar = pd.Series(indices.flatten()).map(df_tf_idf.reset_index()['word'])
    result = pd.DataFrame({'distance':distances.flatten(), 'word': words_similar})
    return result

In [28]:
def most_similar(x, model, top_n=5):
    dists = euclidean_distances(x.reshape(1, -1), model)
    pairs = enumerate(dists[0])
    most_similar = sorted(pairs, key=lambda item: item[1])[:top_n]
    return most_similar

In [29]:
similarities = most_similar(x, lsi_Z)
document_id, similarity = similarities[0]

In [30]:
q_corpus = [stops_removed[-1], stops_removed[-2], stops_removed[-3]]

# unnecessary, as text was already processed 
#processed_corpus = [preprocess_text(doc) for doc in corpus]

q_tfidf_vectorizer = TfidfVectorizer(norm=None)

q_tf_idf_scores = q_tfidf_vectorizer.fit_transform(q_corpus)

q_feature_names = q_tfidf_vectorizer.get_feature_names()
q_corpus_index = [n for n in q_corpus]

q_df_tf_idf = pd.DataFrame(q_tf_idf_scores.T.todense(), index=q_feature_names, columns=q_corpus_index)

In [340]:
tfidf_vec = TfidfVectorizer()

q_drops_vectorized = tfidf_vec.fit_transform(stops_removed)
q_drops_features = tfidf_vec.get_feature_names()
q_drops_index = [line for line in stops_removed]

q_drops_df = pd.DataFrame(q_drops_vectorized.T.todense(), index=q_drops_features, columns=q_drops_index)

NameError: name 'stops_removed' is not defined

In [339]:
q_drops_df.sort_values(by=q_drops_df.columns[-1], ascending=False).T

NameError: name 'q_drops_df' is not defined

In [31]:
q_df_tf_idf.sort_values(by=q_df_tf_idf.columns[2], ascending=False)

,mockingbird hrc detained arrested yet huma follow huma nothing w russia yet potus surround w generals military intelligence go around letter agencies supreme court case allows use mi v congressional assembled approved agencies ultimate authority branches military wo approval conditions unless wartime conditions military code aw held potus go tv address nation potus must isolate prevent negative optics potus knew removing criminal rogue elements first step essential free pass legislation access everything classified believe hrc soros obama etc power trump fantasy whoever controls office presidecy controls great land never believed moment democrats republicans would lose control r v battle soros donate money recently would place funds rc mockingbird god bless fellow patriots,open eyes finally came rodbob key players uranium scandal dont think potus would tweeting removal given clear conflict potus meet bob cover fbi dir interview bob unable serve dir per law gowdy comments comey history potus everything everyone corrupt fewer think follow huma operation mockingbird priority clean bad actors unite people behind america first agenda many govt worship satan republicans v democrats stage hrc ng called across cities trust president god bless patriots,us come drop crumbs crumbs potus insulated discussion suggesting hes even target false potus addressing nation issues people begin indicted must remain neutral pure optical reasons suggest plan false common sense focus military intellingence state secrets might used vs three letter agency sc decision opened door sitting president activate must showed potus surrounded generals lot good people bad faith hostile takeover evil corrupt network players democrats dont fool thinking obama soros roths clintons etc power present day potus operation mockingbird patriots control sit back enjoy show
potus,4.000000,3.000000,4.000000
false,0.000000,0.000000,3.386294
crumbs,0.000000,0.000000,3.386294
must,1.287682,0.000000,2.575364
people,0.000000,1.287682,2.575364
...,...,...,...
funds,1.693147,0.000000,0.000000
given,0.000000,1.693147,0.000000
go,3.386294,0.000000,0.000000
god,1.287682,1.287682,0.000000


In [32]:
chapter_11 = open('/Users/kylereaves/Desktop/conspiratorial_texts/mein_kampf/11_race_and_nation.txt', 'r')
content = chapter_11.read()

In [33]:
eleven_counter = CountVectorizer()

eleven_matrix = eleven_counter.fit_transform([content])

eleven_features = eleven_counter.get_feature_names()

eleven_df = pd.DataFrame(eleven_matrix.T.todense(), index=eleven_features, columns=['Term Frequencies'])

In [34]:
sentences = nltk.sent_tokenize(content)
chapter_11_tokens = [nltk.word_tokenize(sentence) for sentence in sentences]

In [35]:
processed_sentences = [preprocess_text(sentence) for sentence in sentences]

In [36]:
chapter_11_cleaned = [preprocess_text(sentence) for sentence in sentences]

eleven_count = TfidfVectorizer(norm=None)

chapter_11_vectorized = eleven_count.fit_transform(chapter_11_cleaned)
chapter_11_features = eleven_count.get_feature_names()

chapter_11_df = pd.DataFrame(chapter_11_vectorized.T.todense(), index=chapter_11_features, columns=[sent for sent in chapter_11_cleaned])

In [97]:
q_nbrs = NearestNeighbors(n_neighbors=20).fit(chapter_11_vectorized.T)

row = chapter_11_df.index.get_loc('russia')
distances, indices = q_nbrs.kneighbors(chapter_11_vectorized.T.getrow(row))
names_similar = pd.Series(indices.flatten())
result = pd.DataFrame(data={'distance': distances.flatten(), 'name':names_similar, 'topic': [chapter_11_df.index[i] for i in indices.flatten()]})

In [98]:
result

,distance,name,topic
0,0.000000,726,ferocity
1,0.000000,1642,scribbler
2,0.000000,409,crowd
3,0.000000,1614,rulership
4,0.000000,1054,kill
5,0.000000,1774,starve
6,0.000000,991,inhuman
7,0.000000,1601,robber
8,0.000000,1617,russia
9,0.000000,1891,thirty


In [40]:
chapter_11_df.sort_values(by=chapter_11_df.columns[-3], ascending=False)

,statement truth obvious reason common world see least recognize,time world pass well know truism blindly astonish suddenly somebody discover everybody ought know,columbus egg lie hundred thousand columbus rarely see,thus without exception people wander nature garden think know almost everything yet exception walk blindly one outstanding principle nature work inner seclusion specie live be earth,even superficial observation show almost brazen basic principle countless form expression nature live limit form propagation increase limit,every animal mate representative specie,titmouse seek titmouse finch finch stork stork field mouse field mouse common mouse common mouse wolf wolf etc,exceptional circumstance change first compulsion captivity well impossibility mat within specie,nature begin resist help visible mean visible protest consist either deny bastard procreative faculty limit fertility come offspring case take away capacity resistance disease inimical attack,natural,...,activity condemn unproductiveness sole reason favorable case best saw try fight symptomatic form general sickness pass blindly germ,systematically follow political development old reich bind arrive upon quiet examination realization even time unity thus rise german nation inner decay already way despite apparent political success rise economic wealth general situation become bad year year,even election reichstag outward swell marxist vote announce rapidly approach internal external collapse,success call bourgeois party value unable check increase number marxist flood even call bourgeois electoral victory even harbor ferment deterioration,bourgeois world without know infect cadaveric poison marxist idea resistance originate frequently rather competitive envy ambitious leader rejection principle adversary determine fight extreme,one long year fight imperturbable regularity jew,star david rise high high measure people self preservation vanish,therefore august 1914 people determine attack rush battlefield take place last flare national instinct self preservation face progress pacifist marxist paralyzation national body,even fateful day one recognize internal enemy outward resistance vain providence give reward victorious sword follow law eternal revenge,inner realization form u lead principle well tendency new movement alone conviction enable bring decline german people standstill also create granite foundation upon one day exist state represent mechanism economic consideration interest alien people folkish organism germanic state german nation
national,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.710307,0.0,0.0
1914,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.657739,0.0,0.0
rush,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.657739,0.0,0.0
flare,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.252273,0.0,0.0
august,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.252273,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
faint,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
fail,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
faculty,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.657739,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
factory,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0


In [51]:
from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, LabelSet
output_notebook()

Loading BokehJS ...

In [64]:
svd = TruncatedSVD(n_components=2)
documents_2d = svd.fit_transform(chapter_11_vectorized.T)

df = pd.DataFrame(columns=['x', 'y', 'document'])
df['x'], df['y'], df['document'] = documents_2d[:,0], documents_2d[:,1], chapter_11_features

source = ColumnDataSource(ColumnDataSource.from_df(df))
labels = LabelSet(x="x", y="y", text="document", y_offset=8,
                    text_font_size='8pt', text_color='#555555',
                    source=source, text_align='center')

plot = figure(plot_width=600, plot_height=600)
plot.circle("x", "y", size=12, source=source, line_color="black", fill_alpha=0.8)
plot.add_layout(labels)
show(plot, notebook_handle=True)

In [86]:
q_documents_2d = svd.fit_transform(q_drops_vectorized.T)

qdf = pd.DataFrame(columns=['x', 'y', 'document'])
qdf['x'], qdf['y'], qdf['document'] = q_documents_2d[:,0], q_documents_2d[:,1], q_drops_features

q_source = ColumnDataSource(ColumnDataSource.from_df(qdf))
q_labels = LabelSet(x="x", y="y", text="document", y_offset=8,
                    text_font_size='8pt', text_color='#555555',
                    source=q_source, text_align='center')

qplot = figure(plot_width=600, plot_height=600)
qplot.circle("x", "y", size=12, source=q_source, line_color="black", fill_alpha=0.8)
qplot.add_layout(q_labels)
show(qplot, notebook_handle=True)

In [117]:
qdf[qdf.document.str.contains('awakening')]

,x,y,document
709,4.965490e-03,6.292779e-03,awakening
4186,2.960512e-13,7.197815e-14,greatawakening
9574,1.404358e-02,4.249221e-02,thegreatawakening


In [88]:
df

,x,y,document
0,0.380641,-0.209717,1914
1,0.080712,0.038437,1918
2,0.050968,-0.056008,abate
3,3.578097,0.300698,ability
4,4.225683,0.729187,able
...,...,...,...
2111,0.157034,-0.026401,yield
2112,1.189619,-0.072356,young
2113,0.078419,0.034106,zeitung
2114,0.258819,0.105940,zion
